In [83]:
%reload_ext watermark
%watermark -a "Eduardo Teles Guimaraes" --iversions

Author: Eduardo Teles Guimaraes

scipy     : 1.9.1
numpy     : 1.21.5
matplotlib: 3.5.2
seaborn   : 0.11.2
pandas    : 1.4.4



In [84]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [85]:
class Dense:
    '''
    Classe que cria a arquitetura da rede neural:
    entradas
    feat_size :: tamanho de entradas iniciais da arquitetura 1, 2, 3, .., n
    out_size :: geralmente 1 saida, ou entao, multicamadas
    '''
    def __init__(self, input_size, out_size):
        # Tamanho das dimensoes
        self.input_size = input_size
        self.out_size = out_size
        # np.random.normal(0, 1, 3 * 1) * np.sqrt(2 / 3)    array([ 0.40406558, -0.27438716,  0.58409914])
        # Criando um vetor de peso alinhado com a dimensao de entrada e saida
        self.weights = (np.random.normal(0, 1, input_size * out_size) * np.sqrt(2 / input_size)).reshape(input_size, out_size)
        #Avoid negative output
        self.bias = np.random.rand(1, out_size) - 0.5
    
    def forward(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward(self, output_derivative, lr=0.5):
        """
        output_derivative : derivada da saida
        lr: learning_rate
        """
        # erro = output - output_layer
        # Calcular o erro -> (w+1) = (w(n) * erro * lr)
        input_derivative = np.dot(output_derivative, self.weights.T)
        new_weight = np.dot(self.input.T.reshape(-1, 1), output_derivative)

        self.weights -= lr * new_weight
        self.bias -= lr * output_derivative

        return input_derivative

In [86]:
def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    x[x > 0] = 1
    x[x < 0] = 0

    return x

class Activation:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)

        return self.output

    def backward(self, output_derivative, lr):
        return(self.activation_prime(self.input) * output_derivative)

In [87]:
# Mean squared erro
def mse(y_true, y_pred):
    return (np.mean((y_pred - y_true) ** 2))

def mse_derivative(y_true, y_pred):
    return (2 * (y_pred - y_true) / y_true.size)

def binary_cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15  # pequeno valor adicionado para evitar log(0)
    loss = - np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))
    return loss

def binary_cross_entropy_loss_derivative(y_true, y_pred):
    return (y_pred - y_true) / (y_pred * (1 - y_pred))
    
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [92]:
class Network:
    '''
    Inicializando o modelo com a funcao de perda
    Encontrar o melhor valor da funcao de perda ou
    O melhor erro posivel do modelo
    '''
    def __init__(self, loss, loss_prime):
        self.layers = []
        self.loss = loss
        self.loss_prime = loss_prime

    def add(self, layer):
        self.layers.append(layer)

    def predict(self, input_data):

        result = []

        for i in range(len(input_data)):
            layer_output = input_data[i]

            for layer in self.layers:
                layer_output = layer.forward(layer_output)
            result.append(layer_output)

        return result

    def fit(self, x_train, y_train, epochs, lr):
        
        for e in range(epochs):
            error = 0

            for j in range(len(x_train)):

                layer_output = x_train[j]

                for layer in self.layers:
                    layer_output = layer.forward(layer_output)

                error += self.loss(y_train[j], layer_output)

                gradient = self.loss_prime(y_train[j], layer_output)

                for layer in reversed(self.layers):
                    gradient = layer.backward(gradient, lr)

            error /= len(x_train)
            
            print(f"Epoch {e + 1}/{epochs}  Erro={error}")


In [95]:
# Dados
x_train = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# Ajuste dos dados
x_train = x_train.reshape(-1, 2)
y_train = y_train.reshape(-1, 1)

# Modelo
modelo_xor = Network(mse, mse_derivative)
modelo_xor.add(Dense(2, 3))
modelo_xor.add(Activation(sigmoid, sigmoid_prime))
modelo_xor.add(Dense(3, 1))

# Treinamento
modelo_xor.fit(x_train, y_train, epochs = 1000, lr = 0.05)

# Teste
y_pred = modelo_xor.predict(x_train)

Epoch 1/1000  Erro=0.8316830485963578
Epoch 2/1000  Erro=0.49156009824817853
Epoch 3/1000  Erro=0.43779880555687234
Epoch 4/1000  Erro=0.4244343151253427
Epoch 5/1000  Erro=0.4172980614077935
Epoch 6/1000  Erro=0.4113542664587955
Epoch 7/1000  Erro=0.40583914675597743
Epoch 8/1000  Erro=0.40062627080597396
Epoch 9/1000  Erro=0.39568363987088884
Epoch 10/1000  Erro=0.39099320163140805
Epoch 11/1000  Erro=0.38653946121106786
Epoch 12/1000  Erro=0.3823080643818123
Epoch 13/1000  Erro=0.3782856579556494
Epoch 14/1000  Erro=0.37445984888256123
Epoch 15/1000  Erro=0.37081914506111147
Epoch 16/1000  Erro=0.36735288501726115
Epoch 17/1000  Erro=0.36405116694100426
Epoch 18/1000  Erro=0.36090478189048697
Epoch 19/1000  Erro=0.35790515272296736
Epoch 20/1000  Erro=0.35504427893151536
Epoch 21/1000  Erro=0.35231468705289715
Epoch 22/1000  Erro=0.34970938615305747
Epoch 23/1000  Erro=0.34722182787407263
Epoch 24/1000  Erro=0.3448458705532382
Epoch 25/1000  Erro=0.34257574696721327
Epoch 26/1000  E

In [101]:
print("Valor Real:", "\n",
      list(y_train.reshape(-1,)), "\n",
      "------------", "\n",
      "Valor Previsto:", "\n",
      [round(float(a)) for a in y_pred], "\n",
      [(float(a)) for a in y_pred])

Valor Real: 
 [0, 1, 1, 0] 
 ------------ 
 Valor Previsto: 
 [0, 1, 1, 0] 
 [0.02025037790598977, 0.9863785087536547, 0.9505846433249698, 0.03173000363814277]
